In [ ]:
import sys
sys.path.append('../')

# Loads all methods and data for testing
from DifDifSeg_methods import *

In [ ]:
#Load data
current_directory = os.environ['HOME']
train_data_path = os.path.join(current_directory, 'mri-infarct-segmentation/data/DUPONT/datasplit/FixedPath/train_data.csv')
df_train = pd.read_csv(train_data_path)
train_symDif_data_path = os.path.join(current_directory, 'mri-infarct-segmentation/Anders/V2/Dataset/SymDif_DWI_DUPONT/train.csv')
df_train_symDif = pd.read_csv(train_symDif_data_path)

data = []
for i in range(0, len(df_train)):
    tmp = df_train["DWI_path"][i]
    subject = os.path.join(*(tmp.split('/')[4:5]))

    for j in range(len(df_train)):
        tmp = df_train_symDif["symDif_path"][j]
        subject2 = os.path.join(*(tmp.split('/')[9:10]))
        if subject2 == subject:
            symDif_path = tmp

    data.append([os.path.join(current_directory, df_train["DWI_path"][i]),
                os.path.join(current_directory, df_train["ADC_path"][i]),
                os.path.join(current_directory, df_train["b0"][i]),
                os.path.join(current_directory, df_train["Label_path"][i]),
                symDif_path])

dataset = CustomDataset(data, "train")

In [ ]:
#Training
nEpoch = 15
lr = 1e-6
batchSize = 1
model = CustomModel()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
seg_loss = monai.losses.DiceLoss(reduction='mean')
c = torch.nn.L1Loss(reduction='mean')
trainDataLoader = DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=0)

run = neptune.init_run(
    project="MRI-infarct-segmentation/DifDifSeg",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkODc2ZGIzZC0wMjllLTQ2OGEtODlmZC05MGNmZDg2YzYwMWUifQ==",
)
params = {"Data": "JAS", "Model": "DifDifSeg","Learning_rate": lr, "Optimizer": "Adam"}
run["parameters"] = params

best_diceLoss = 1
model.train()
for iEpoch in range(nEpoch):
    total_train_loss = 0
    total_train_Diceloss = 0
    for DWI_vol, ADC_vol, b0_vol,label_vol, symDif_vol in tqdm(trainDataLoader):
        optimizer.zero_grad() #reset gradients

        histograms = get_histo(DWI_vol.squeeze(0))
        ggg = model(histograms)
        ggg = ggg.reshape((batchSize, dim, 6))

        _symDif, _modDif, _output_segmentation = torch.tensor(segVol(DWI_vol.squeeze(0), ADC_vol.squeeze(0), b0_vol.squeeze(0), symDif_vol.squeeze(0), ggg))

        diceLoss = seg_loss(_output_segmentation, label_vol.squeeze(0))
        total_train_Diceloss += diceLoss.item()
        run["Training/train_Diceloss"].append(diceLoss.item())
        
        ggg_dif = ggg * (2-pow(2,diceLoss.item()))
        loss = c(ggg, ggg_dif)
        loss.backward() #calculate gradients
        optimizer.step() #update weights

        total_train_loss += loss.item()
        run["Training/train_L1loss"].append(loss.item())

        if (iEpoch) % 3 == 0:
            for i in range(13,15):
                if torch.sum(_output_segmentation[:,:,i]) > 0 or torch.sum(label_vol[0,:,:,i]) > 0:
                    fig, axes = plt.subplots(1, 6, figsize = (12, 12))
                    axes[0].imshow(ADC_vol[0,:,:,i], cmap='gray')
                    axes[0].set_title('ADC')
                    axes[1].imshow(DWI_vol[0,:,:,i], cmap='gray')
                    axes[1].set_title('DWI')
                    axes[2].imshow(_symDif[:,:,i], cmap='gray')
                    axes[2].set_title('symDif')
                    axes[3].imshow(_modDif[:,:,i], cmap='gray')
                    axes[3].set_title('modDif')
                    axes[4].imshow(_output_segmentation[:,:,i], cmap='gray')
                    axes[4].set_title('dif_dif')
                    axes[5].imshow(label_vol[0,:,:,i], cmap='gray')
                    axes[5].set_title('label (GT)')
                    for ax in axes:
                        ax.axis('off')
                    plt.show()
    train_loss_avg = total_train_loss / len(trainDataLoader)
    train_Diceloss_avg = total_train_Diceloss / len(trainDataLoader)
    print('Total Diceloss:',round(train_Diceloss_avg,3))
    print('Total L1loss:',round(train_loss_avg,3))
    run["Training/epoch_train_loss"].append(train_loss_avg)
    run["Training/epoch_train_Diceloss"].append(train_Diceloss_avg)

    if train_Diceloss_avg < best_diceLoss:
        best_diceLoss = train_Diceloss_avg
        torch.save(model.state_dict(), '/home/rosengaard/mri-infarct-segmentation/Anders/Figures_for_rapport/SegMethods/DifDifSeg/Training/weights/DifDifSeg_model_Dummy.pt')
        print('New model saved!')
print('______________________________________________')
run.stop()
print('Training ended')